# Download ligand

In [ ]:
from rcsbapi.search import search_attributes as attrs

ECnumber = "3.4.21.4" # look for suitable ligand from https://www.brenda-enzymes.org/index.php

q1 = attrs.rcsb_polymer_entity.rcsb_ec_lineage.id == ECnumber
# Use 'range' for ligand molecular weight (between 300 and 800)
q2 = attrs.chem_comp.formula_weight >= 300
q3 = attrs.chem_comp.formula_weight <= 800

query = q1 & q2 & q3
resultL = list(query())

print("☑️ There are", len(resultL), "trypsin structures that contain ligands in the RCSB PDB.")

In [ ]:
molResultL = list(query("mol_definition"))
print("☑️ There are",len(molResultL), "ligands for EC Number", ECnumber, "in this list. Here is a list of the first 10 ligands.")
print("First 10 ligand: ", molResultL[0:10])

In [ ]:
import requests #pull files from the PDB
import os

os.makedirs("ligands", exist_ok=True)

baseUrl = "https://files.rcsb.org/ligands/download/"

for ChemID in molResultL:
    cFile = f"{ChemID}_ideal.sdf"
    cFileUrl = baseUrl + cFile
    cFileLocal = "ligands/" + cFile
    response = requests.get(cFileUrl)
    with open(cFileLocal, "w+") as file:
        file.write(response.text)

print(f"☑️ Downloaded {len(molResultL)} ligands.")

# Check and modify ligand

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

# Configuration for displaying in Jupyter notebooks
IPythonConsole.ipython_useSVG = True  # Use SVG for higher quality images
IPythonConsole.drawOptions.addAtomIndices = True  # Show atom indices
IPythonConsole.molSize = 600,600 # Set size of image

ligand = Chem.MolFromMolFile("ligands/13U_ideal.sdf") #load ideal 13U ligand
ligand

In [ ]:
# load a duplicate copy of 13U to manipulate
mod_ligand_N = Chem.MolFromMolFile("ligands/13U_ideal.sdf")

# change carbon (position 23) in ring into nitrogen
mod_ligand_N.GetAtomWithIdx(23).SetAtomicNum(7)
mod_ligand_N

In [ ]:
# Optimize new molecules and save
from rdkit.Chem import AllChem

Chem.SanitizeMol(mod_ligand_N)
mod_ligand_NH = Chem.AddHs(mod_ligand_N)

# Do a constrained embedding to keep the ligand in the same position
# this allows for the hydrogens to be added in reasonable locations, but keeps
# the heavy atoms in the same position
# See https://rdkit.org/docs/source/rdkit.Chem.AllChem.html#rdkit.Chem.AllChem.ConstrainedEmbed
constrained_mol = AllChem.ConstrainedEmbed(mod_ligand_NH, mod_ligand_N, useTethers=True)
constrained_mol

In [ ]:
# Perform geometry optimization
opt_N = AllChem.MMFFOptimizeMolecule(mod_ligand_NH)
mod_ligand_NH

In [ ]:
# save to new files
import os

# make modified ligand directory
os.makedirs("ligands_to_dock", exist_ok=True)

ligand_H = Chem.MolFromMolFile("ligands/13U_ideal.sdf", removeHs=False)

# save modified ligands sdf file - make sure all contain hydrogens and place 
# in a folder of ligands to dock.
Chem.MolToMolFile(ligand_H, 'ligands_to_dock/13U.sdf')
Chem.MolToMolFile(mod_ligand_NH, 'ligands_to_dock/13U_modified_N.sdf')

# Download protein and prepare

In [ ]:
from rcsbapi.search import search_attributes as attrs
from rcsbapi.search import TextQuery

# 1. Define the search criteria
ECnumber = "3.4.21.4" 

q1 = attrs.rcsb_polymer_entity.rcsb_ec_lineage.id == ECnumber
q2 = TextQuery("13U")  # Trypsin has ligand 13U

query = q1 & q2 
results = list(query())

print("☑️ PDB ID that have trypsins and 13U:", results)

In [ ]:
import os # for making directories
import requests

pdb_id = results[0].lower() #lower case the first result
pdb_id

protein_directory = "protein_structures"
os.makedirs(protein_directory, exist_ok=True)

pdb_request = requests.get(f"https://files.rcsb.org/download/{pdb_id}.pdb")
pdb_request.status_code #200 means ok

In [ ]:
with open(f"{protein_directory}/{pdb_id}.pdb", "w+") as f:
    f.write(pdb_request.text)

In [ ]:
import MDAnalysis as mda
u = mda.Universe(f"{protein_directory}/{pdb_id}.pdb")
u

In [ ]:
import nglview as nv
view = nv.show_mdanalysis(u)
view

In [ ]:
# Select protein atoms
protein = u.select_atoms("protein")
ligand = u.select_atoms("resname 13U")
water = u.select_atoms("resname HOH")

water

In [ ]:
view = nv.show_mdanalysis(protein)
view.clear_representations()
## Add representation to view protein surface colored by hydrophobicity
view.add_representation("surface", colorScheme="hydrophobicity")
lig_view = view.add_component(ligand)
lig_view.add_representation("ball+stick")
water_view = view.add_component(water)
water_view.add_representation("spacefill")
view

In [ ]:
# Write protein to new PDB file
protein.write(f"{protein_directory}/protein_{pdb_id}.pdb")

In [ ]:
! pdb2pqr --pdb-output=protein_structures/protein_h.pdb --pH=7.4 protein_structures/protein_2zq2.pdb protein_structures/protein_2zq2.pqr --whitespace

In [ ]:
pdbqt_directory = "pdbqt"
os.makedirs(pdbqt_directory, exist_ok=True)

u = mda.Universe(f"{protein_directory}/protein_{pdb_id}.pqr")
u.atoms.write(f"{pdbqt_directory}/{pdb_id}.pdbqt")

In [ ]:
# Read in the just-written PDBQT file, replace text, and write back
with open(f"{pdbqt_directory}/{pdb_id}.pdbqt", 'r') as file:
    file_content = file.read()

# Replace 'TITLE' and 'CRYST1' with 'REMARK'
file_content = file_content.replace('TITLE', 'REMARK').replace('CRYST1', 'REMARK')

# Write the modified content back to the file
with open(f"{pdbqt_directory}/{pdb_id}.pdbqt", 'w') as file:
    file.write(file_content)

In [ ]:
!C:\Users\deann\anaconda3\Scripts\mk_prepare_ligand.exe -i ligands_to_dock/13U.sdf -o pdbqt/13U.pdbqt

In [ ]:
!C:\Users\deann\anaconda3\Scripts\mk_prepare_ligand.exe -i ligands_to_dock/13U_modified_N.sdf -o pdbqt/13U_modified_N.pdbqt

# Docking

In [ ]:
# Download vina.exe and vina_split.exe in the same folder from https://github.com/ccsb-scripps/AutoDock-Vina/releases
!vina --version # check if everything correct

In [ ]:
# find the center of the ligand
import MDAnalysis as mda

original_structure = mda.Universe("protein_structures/2zq2.pdb")
ligand_mda = original_structure.select_atoms("resname 13U")

# Get the center of the ligand as the "pocket center"
pocket_center = ligand_mda.center_of_geometry()
print(pocket_center) #3D coordinates of the center of the box

In [ ]:
# compute min and max coordinates of the ligand
# take the ligand box to be the difference between the max and min in each direction.
ligand_box = ligand_mda.positions.max(axis=0) - ligand_mda.positions.min(axis=0)
ligand_box

In [ ]:
pocket_center = pocket_center.tolist()
ligand_box = ligand_box.tolist() ## convert ligand_box to list

In [ ]:
import os

pdb_id = "2zq2"
ligand = "13U"

os.makedirs("docking_results", exist_ok=True)

In [ ]:
!vina.exe --receptor {"pdbqt/" + pdb_id + ".pdbqt"} --ligand {"pdbqt/" + ligand + ".pdbqt"} --center_x {pocket_center[0]} --center_y {pocket_center[1]} --center_z {pocket_center[2]} --size_x {ligand_box[0]} --size_y {ligand_box[1]} --size_z {ligand_box[2]} --exhaustiveness 5 --num_modes 5 --out docking_results/{ligand}_out.pdbqt
# increase exhaustiveness to 32 for better results

In [ ]:
# This creates files like ligand_out_ligand_1.pdbqt, ligand_out_ligand_2.pdbqt, etc.
!vina_split.exe --input {"docking_results/" + ligand + "_out.pdbqt"}

In [ ]:
import subprocess
import re

# We assume you asked for 5 modes in your docking command
for i in range(1, 6):
    pose_file = f"docking_results/{ligand}_out_ligand_{i}.pdbqt"
    
    # Construct the command
    cmd = [
        "vina.exe",
        "--receptor", f"pdbqt/{pdb_id}.pdbqt",
        "--ligand", pose_file,
        "--center_x", str(pocket_center[0]),
        "--center_y", str(pocket_center[1]),
        "--center_z", str(pocket_center[2]),
        "--size_x", str(ligand_box[0]),
        "--size_y", str(ligand_box[1]),
        "--size_z", str(ligand_box[2]),
        "--score_only"
    ]
    
    # Run and capture output
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    # Print the part of the output that contains the energy table
    print(f"--- Energy for Pose {i} ---")
    print(result.stdout)

In [ ]:
import subprocess
import re
import pandas as pd
import os

# Your settings
ligand = "13U"
pdb_id = "2zq2"
all_energies = []

for i in range(1, 6):
    pose_file = f"docking_results/{ligand}_out_ligand_{i}.pdbqt"
    if not os.path.exists(pose_file):
        continue

    # Run the scoring command
    cmd = [
        "vina.exe", "--receptor", f"pdbqt/{pdb_id}.pdbqt", "--ligand", pose_file,
        "--center_x", str(pocket_center[0]), "--center_y", str(pocket_center[1]), "--center_z", str(pocket_center[2]),
        "--size_x", str(ligand_box[0]), "--size_y", str(ligand_box[1]), "--size_z", str(ligand_box[2]),
        "--score_only"
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    output = result.stdout

    # Extract values using regex based on the output you provided
    free_energy = re.search(r"Estimated Free Energy of Binding\s+:\s+([-+]?\d*\.\d+|\d+)", output)
    inter_energy = re.search(r"\(1\) Final Intermolecular Energy\s+:\s+([-+]?\d*\.\d+|\d+)", output)
    intra_energy = re.search(r"\(2\) Final Total Internal Energy\s+:\s+([-+]?\d*\.\d+|\d+)", output)
    torsion_energy = re.search(r"\(3\) Torsional Free Energy\s+:\s+([-+]?\d*\.\d+|\d+)", output)

    if free_energy and inter_energy and intra_energy and torsion_energy:
        all_energies.append({
            'Pose': i,
            'Free Energy (kcal/mol)': float(free_energy.group(1)),
            'Inter': float(inter_energy.group(1)),
            'Intra': float(intra_energy.group(1)),
            'Torsional': float(torsion_energy.group(1))
        })

# Create DataFrame
df = pd.DataFrame(all_energies)

# Save to CSV
output_path = "docking_results/13U_energies.csv"
df.to_csv(output_path, index=False)

print(f"Success! Data saved to {output_path}")
print(df)

In [ ]:
!C:\Users\deann\anaconda3\Scripts\mk_export.exe docking_results/13U_out.pdbqt -s docking_results/13U.sdf

In [ ]:
import prolif as plf
import MDAnalysis as mda

pdb_id = "2zq2"

protein = mda.Universe(f"protein_structures/protein_h.pdb")

In [ ]:
protein_plf = plf.Molecule.from_mda(protein)
poses_plf = plf.sdf_supplier("docking_results/13U.sdf")

In [ ]:
fp = plf.Fingerprint(count=True)
# run on your poses
fp.run_from_iterable(poses_plf, protein_plf)

In [ ]:
pose_index=1
fp.plot_lignetwork(poses_plf[pose_index])

In [ ]:
view = fp.plot_3d(
    poses_plf[pose_index], protein_plf, frame=pose_index, display_all=False
)
view